In [17]:
# Dependencies
import pandas as pd
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Read in candidates data
df1 = pd.read_csv("candidates_clean.csv")
df_cand = df1.copy()
df_cand.head()

,Unnamed: 0,id,cycle,fec_cand_id,cid,first_last_party,party,dist_id_run_for,dist_id_currently_held,current_candidate,cycle_candidate,crpico,recip_code,nopacs,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized,result
0,0,56756,1996,P80001118,N00000948,Lenora B Fulani (3),3,PRES,,,,C,3L,,0.0,6000.0,6000,0.0,L
1,1,58068,1996,S6MN00143,N00004451,Stephen Bonsal Young (R),R,MNS1,,,Y,C,RL,,492.0,97408.0,264254,166354.0,L
2,2,57382,1996,S6DE00073,N00001670,Raymond J Clatworthy (R),R,DES2,,Y,Y,C,RL,,202815.0,618935.0,1636414,814664.0,L
3,3,57290,1996,S4VT00066,N00000525,Gavin Terence Mills (I),U,VTS1,,,,C,3L,,0.0,266.0,266,0.0,L
4,4,53386,1996,H2OK05045,N00005551,Ernest J Istook (R),R,OK05,OK05,Y,Y,I,RW,,130384.0,99600.0,399980,169996.0,W


In [3]:
# Read in pac data 
# df2 = pd.read_csv("pacs.csv", on_bad_lines='skip')
# df_pac = df2.copy()
# df_pac.head()

# Cleaned
df2 = pd.read_csv('pacs_clean.csv')
df_pac = df2.copy()
df_pac.head()

,Unnamed: 0,id,cycle,fec_rec_no,pac_id,cid,amount,date,real_code,type,di,fec_cand_id
0,0,1,2000,1047706,C00147066,N00005892,0.0,2000-04-12 00:00:00,F3300,24K,D,H4TX22023
1,1,2,2000,1470335,C00139964,N00007879,0.0,2000-08-01 00:00:00,A1500,24K,D,S0WA00019
2,2,3,2000,1469635,C00139964,N00006246,0.0,2000-08-01 00:00:00,A1500,24K,D,S4WY00055
3,3,4,2000,364522,C00089086,N00002898,1.0,1999-07-20 00:00:00,H1700,24Z,D,H4FL16021
4,4,5,2000,1786857,C00089086,N00005638,1.0,2000-09-12 00:00:00,H1700,24Z,D,H4TX26016


## Clean

In [4]:
# Handle nulls
# df_pac.isnull().sum()

In [5]:
# df_pac = df_pac.dropna()

In [6]:
# df_pac.duplicated().sum()

In [7]:
# Drop duplicates
# df_pac = df_pac.drop_duplicates()

In [8]:
# df_pac.to_csv('pacs_clean.csv')

In [9]:
# # Join pac and candidates dfs
# df_paccand = df_cand.merge(df_pac, how='left', on='cid')
# df_paccand.head()

## Explore Features

In [10]:
# See if cycle_candidate is significant
# if there are W and blank c_c, then it's significant
# if there L for every blank c_c, insignificant - drop
df3 = df_cand[(df_cand['cycle_candidate'] == ' ') & (df_cand['result'] == 'W')]
df3.head(10)

# KEEP for now, but test both ways.

,Unnamed: 0,id,cycle,fec_cand_id,cid,first_last_party,party,dist_id_run_for,dist_id_currently_held,current_candidate,cycle_candidate,crpico,recip_code,nopacs,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized,result
36,36,57183,1996,S4MO00037,N00005165,John Ashcroft (R),R,MOS2,MOS2,,,I,RW,,-20076.0,76499.0,131498,75075.0,W
45,45,57578,1996,P60003654,N00008333,Newt Gingrich (R),R,GA06,GA06,,,C,RW,,1096218.0,1843635.0,8034113,5094260.0,W
546,546,46102,1992,P00000232,N00004698,Phil Crane (R),R,IL08,IL12,,,I,RW,Y,18624.0,138804.0,608766,451338.0,W
601,601,37868,1990,H0ID01030,N00002091,Larry Craig (R),R,IDS1,ID01,,,O,RW,,1125715.0,396203.0,2269861,747943.0,W
612,612,37928,1990,H0IN04022,N00003845,Dan Coats (R),R,INS2,INS2,,,I,RW,,1891769.0,1547097.0,5846249,2407383.0,W
646,646,50703,1994,S2NC00042,N00002327,Sue Myrick (R),R,NC09,,,,O,RW,,214300.0,322810.0,537110,0.0,W
752,752,52444,1996,H0OR03026,N00007724,Ron Wyden (D),D,ORS2,ORS2,,,O,DW,,2133242.0,2026828.0,6553454,2393384.0,W
768,768,55739,1996,H8WY00106,N00006246,Craig Thomas (R),R,WYS1,WYS1,,,I,RW,,23414.0,4800.0,33720,5506.0,W
779,779,57858,1996,S6PA00100,N00001604,Arlen Specter (R),R,PAS1,PAS1,,,I,RW,,364126.0,2523128.0,3846352,959098.0,W
784,784,57717,1996,P80000581,N00005037,Richard A Gephardt (D),D,MO03,MO03,,,I,DW,,1254013.0,1751202.0,4226115,1220900.0,W


In [11]:
# Choose columns for features
columns = ['party', 'current_candidate', 'cycle_candidate', 'crpico', 'raised_from_pacs', 'raised_from_individuals',	'raised_total',	'raised_unitemized']

# # consider dropping later:
#   #cycle_candidate
#   #nopacs
#   #dist_id_currently_held

df_features = df_cand[columns]
df_features.head()

,party,current_candidate,cycle_candidate,crpico,nopacs,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized
0,3,,,C,,0.0,6000.0,6000,0.0
1,R,,Y,C,,492.0,97408.0,264254,166354.0
2,R,Y,Y,C,,202815.0,618935.0,1636414,814664.0
3,U,,,C,,0.0,266.0,266,0.0
4,R,Y,Y,I,,130384.0,99600.0,399980,169996.0


In [12]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19863 entries, 0 to 19862
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   party                    19863 non-null  object 
 1   current_candidate        19863 non-null  object 
 2   cycle_candidate          19863 non-null  object 
 3   crpico                   19863 non-null  object 
 4   nopacs                   19863 non-null  object 
 5   raised_from_pacs         19863 non-null  float64
 6   raised_from_individuals  19863 non-null  float64
 7   raised_total             19863 non-null  int64  
 8   raised_unitemized        19863 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 1.4+ MB


In [13]:
df_features.nunique()

party                          4
current_candidate              2
cycle_candidate                2
crpico                         5
nopacs                         2
raised_from_pacs           11607
raised_from_individuals    15530
raised_total               17987
raised_unitemized          14653
dtype: int64

## Preprocessing

In [21]:
############ BUCKET categorical vars ############
for col in df_features.columns:
  print(df_features[col].value_counts())

# drop nopacs, bucket crpico 

R    9746
D    8779
U     786
3     552
Name: party, dtype: int64
Y    12224
      7639
Name: current_candidate, dtype: int64
Y    17737
      2126
Name: cycle_candidate, dtype: int64
C    9506
I    5675
O    4596
       85
3       1
Name: crpico, dtype: int64
     19668
Y      195
Name: nopacs, dtype: int64
0.0         4435
1000.0       343
500.0        243
5000.0       157
2000.0       147
            ... 
570.0          1
421654.0       1
435193.0       1
10098.0        1
345241.0       1
Name: raised_from_pacs, Length: 11607, dtype: int64
 0.0          865
 1000.0       129
 500.0        124
 250.0         64
 200.0         54
             ... 
 5616.0         1
 153472.0       1
 9658.0         1
 1154484.0      1
-127.0          1
Name: raised_from_individuals, Length: 15530, dtype: int64
 500        107
 1000        79
 200         42
 250         41
 300         32
           ... 
 2260         1
 27975        1
 969956       1
 8680358      1
-127          1
Name: raised_total

In [24]:
# get value counts variable
crpico_type = df_features['crpico'].value_counts()
crpico_type

C    9506
I    5675
O    4596
       85
3       1
Name: crpico, dtype: int64

In [25]:
# Determine which values to replace if counts are less than ..?
replace_crpico = list(crpico_type[crpico_type < 100].index)

# Replace in dataframe
for cls in replace_crpico:
    df_features.crpico = df_features.crpico.replace(cls,"Other")
    
# Check to make sure binning was successful
df_features.crpico.value_counts()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


C        9506
I        5675
O        4596
Other      86
Name: crpico, dtype: int64

In [16]:
########### ENCODE VARIABLES #############

# generate categorical variable list
catvars = df_features.dtypes[df_features.dtypes == 'object'].index
catvars

Index(['party', 'current_candidate', 'cycle_candidate', 'crpico', 'nopacs'], dtype='object')

In [19]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_features[catvars]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(catvars)
encode_df.head()

,party_3,party_D,party_R,party_U,current_candidate_,current_candidate_Y,cycle_candidate_,cycle_candidate_Y,crpico_,crpico_3,crpico_C,crpico_I,crpico_O,nopacs_,nopacs_Y
0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [20]:
# Merge one-hot encoded features and drop the originals
encoded_df = df_features.merge(encode_df, left_index=True, right_index=True)
encoded_df = encoded_df.drop(catvars, axis=1)
encoded_df.head()

,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized,party_3,party_D,party_R,party_U,current_candidate_,current_candidate_Y,cycle_candidate_,cycle_candidate_Y,crpico_,crpico_3,crpico_C,crpico_I,crpico_O,nopacs_,nopacs_Y
0,0.0,6000.0,6000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,492.0,97408.0,264254,166354.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,202815.0,618935.0,1636414,814664.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,266.0,266,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,130384.0,99600.0,399980,169996.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
